# Formatting Tables in Pandas


In [45]:
# Import pandas
import pandas as pd
# Import numpy
import numpy as np



## The data
Let's create some simulated data for two widgets, A and B. We'll create a dataframe for each widget, then concatenate them together. We'll also sort the dataframe by month and reset the index.

In [46]:
# simulated data for widget A
df_a = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 1_000_000,
            high = 2_500_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 300_000,
            high = 500_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 750_000,
            high = 1_250_000,
            size = 132
        )
    }
)

df_a['Product'] = 'A'

# simulated data for widget B
df_b = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 100_000,
            high = 800_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 10_000,
            high = 95_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 450_000,
            high = 750_000,
            size = 132
        )
    }
)

df_b['Product'] = 'B'

# put it together & sort
df = pd.concat([df_a,df_b],axis = 0)
df.sort_values(by = 'Month',inplace = True)
df.reset_index(drop = True,inplace = True)

Let’s calculate a few “interesting” statistics — average sale amounts and product conversion:

In [41]:
# average sale
df['Average sale'] = df['Amounts'] / df['Numbers']

# conversion
df['Product conversion'] = df['Numbers'] / df['Quotes']

In [42]:
df.head()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1429350,356382,1030065,A,2.890340,0.249332
1,2012-01-01,284816,87792,597372,B,6.804401,0.308241
2,2012-02-01,1657341,488301,1215097,A,2.488418,0.294629
3,2012-02-01,489414,40396,557960,B,13.812259,0.082540
4,2012-03-01,1482387,359118,1175337,A,3.272843,0.242257


## Date Formatting
There’s arguably nothing __wrong__ with the formatting, but it could be better. For instance, since all the monthly data is reflected as at the first of each month, there’s probably little sense in keeping the day element of each Month entry as it tells the reader very little.

In [47]:
# format the date as YYYY-MM-DD
df.style.format({'Month':'{:%Y-%m}'})
display(df.head(3))

,Month,Quotes,Numbers,Amounts,Product
0,2012-01-01,1504962,312134,952693,A
1,2012-01-01,498961,62262,481539,B
2,2012-02-01,2490291,442022,1112923,A


Now, we can improve readability even further by using the name of each month rather than the month number, and we can do this __*without having to alter the underlying data*__.

In [44]:
# format the date as YYYY-MM-DD
df.style.format({'Month':'{:%B %Y}'})
display(df.iloc[:3])

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1429350,356382,1030065,A,2.890340,0.249332
1,2012-01-01,284816,87792,597372,B,6.804401,0.308241
2,2012-02-01,1657341,488301,1215097,A,2.488418,0.294629
